In [1]:
import pandas as pd, numpy as np
import lbwsg
import lbwsg_boe
import lsff_interventions

!whoami
!date

ndbs
Fri Dec  4 14:25:30 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

# Test updated IronBirthweightCalculator

This version is rewritten to use the new `global_data` and `local_data` tuples in `lsff_interventions`.

In [3]:
artifact_dir = '/share/costeffectiveness/artifacts/vivarium_conic_lsff/'
artifact_path = f'{artifact_dir}/nigeria.hdf'

In [4]:
draws = [0,50,100,200,500]

In [6]:
calc = lbwsg_boe.IronBirthweightCalculator('Nigeria', artifact_path, 2017, draws)

In [9]:
calc.do_back_of_envelope_calculation(1000)

In [10]:
calc.baseline_pop.head()

sex  age  age_start   age_end  \
draw simulant_id                                     
0    0            Female  0.0        0.0  0.019178   
     1              Male  0.0        0.0  0.019178   
     2              Male  0.0        0.0  0.019178   
     3            Female  0.0        0.0  0.019178   
     4              Male  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
0    0                                 0.281173   
     1                                 0.435283   
     2                                 0.963189   
     3                                 0.463808   
     4                                 0.035700   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
0    0                                       0.247674     cat54   
     1                                       0.474490     cat44   
     2                                       0.646328     cat48   
     3                                       0.610502     cat44   
     4                                       0.074445     cat48   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
0    0                 0.254332       0.755379        38.508665  3877.689587   
     1                 0.048284       0.417986        38.096568  2708.993068   
     2                 0.193346       0.535348        37.193346  3267.674165   
     3                 0.897640       0.329967        39.795280  2664.983632   
     4                 0.188929       0.253315        37.188929  3126.657423   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    0                              3877.125070                     False   
     1                              2708.428550                     False   
     2                              3267.109648                     False   
     3                              2664.419115                     False   
     4                              3126.092905                      True   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    0                      0.000000           0.000000          3877.125070   
     1                      0.000000           0.000000          2708.428550   
     2                      0.000000           0.000000          3267.109648   
     3                      0.000000           0.000000          2664.419115   
     4                     23.078027           1.154477          3127.247383   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    0                       cat54             1.783182  
     1                       cat44             2.880156  
     2                       cat48             2.074650  
     3                       cat44             2.557075  
     4                       cat48             2.074650

In [11]:
calc.intervention_pop.head()

sex  age  age_start   age_end  \
draw simulant_id                                     
0    0            Female  0.0        0.0  0.019178   
     1              Male  0.0        0.0  0.019178   
     2              Male  0.0        0.0  0.019178   
     3            Female  0.0        0.0  0.019178   
     4              Male  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
0    0                                 0.281173   
     1                                 0.435283   
     2                                 0.963189   
     3                                 0.463808   
     4                                 0.035700   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
0    0                                       0.247674     cat54   
     1                                       0.474490     cat44   
     2                                       0.646328     cat48   
     3                                       0.610502     cat44   
     4                                       0.074445     cat48   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
0    0                 0.254332       0.755379        38.508665  3877.689587   
     1                 0.048284       0.417986        38.096568  2708.993068   
     2                 0.193346       0.535348        37.193346  3267.674165   
     3                 0.897640       0.329967        39.795280  2664.983632   
     4                 0.188929       0.253315        37.188929  3126.657423   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    0                              3877.125070                     False   
     1                              2708.428550                     False   
     2                              3267.109648                     False   
     3                              2664.419115                     False   
     4                              3126.092905                      True   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    0                      0.000000           0.000000          3877.125070   
     1                      0.000000           0.000000          2708.428550   
     2                      0.000000           0.000000          3267.109648   
     3                      0.000000           0.000000          2664.419115   
     4                     23.078027           1.154477          3127.247383   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    0                       cat54             1.783182  
     1                       cat44             2.880156  
     2                       cat48             2.074650  
     3                       cat44             2.557075  
     4                       cat48             2.074650

In [12]:
calc.population_impact_fraction

draw
0      0.000969
50     0.001128
100   -0.000137
200    0.000000
500    0.000135
Name: lbwsg_relative_risk, dtype: float64

# Why is the PIF still negative sometimes, even after implementing CRN?

## How is it possible that the relative risk increases in the intervention scenario?

## Can increasing someone's birthweight push them into a higher risk category?

Yes, that appears to be what is happening. See below.

### First confirm that mean RR is higher in intervention than baseline, which is opposite of what we expect:

In [13]:
idx = pd.IndexSlice
bp, ip = calc.baseline_pop, calc.intervention_pop
print(bp.loc[idx[100,:], 'lbwsg_relative_risk'].mean())
print(ip.loc[idx[100,:], 'lbwsg_relative_risk'].mean())

10.581696489076958
10.583151260104904


## Now check whether everyone's birthweight actually increased in intervention scenario

Yes, everyone's birthweight either went up or stayed the same.

In [14]:
bp.loc[bp.treated_birthweight > ip.treated_birthweight]

,,sex,age,age_start,age_end,iron_fortification_propensity,mother_flour_consumption_propensity,lbwsg_cat,ga_propensity,bw_propensity,gestational_age,birthweight,treatment_deleted_birthweight,mother_is_iron_fortified,mother_daily_flour,birthweight_shift,treated_birthweight,treated_lbwsg_cat,lbwsg_relative_risk
draw,simulant_id,,,,,,,,,,,,,,,,,,


In [16]:
print(bp.loc[bp.treated_birthweight > ip.treated_birthweight].shape)
print(bp.loc[bp.treated_birthweight < ip.treated_birthweight].shape)
print(bp.loc[bp.treated_birthweight == ip.treated_birthweight].shape)

(0, 18)
(847, 18)
(4153, 18)


In [17]:
847+4153

5000

## Now see how RRs changed between baseline and intervention

4 went down as expected, but 2 went up!

Note that this is for all draws. The results by draw are:

- draw 0: 1 down
- draw 50: 1 down
- draw 100: 1 down, 2 up
- draw 500: 1 down

In [18]:
print(bp.loc[bp.lbwsg_relative_risk > ip.lbwsg_relative_risk].shape)
print(bp.loc[bp.lbwsg_relative_risk < ip.lbwsg_relative_risk].shape)
print(bp.loc[bp.lbwsg_relative_risk == ip.lbwsg_relative_risk].shape)

(4, 18)
(2, 18)
(4994, 18)


### Simulants whose RR went down (expected direction):

In [19]:
bp.loc[bp.lbwsg_relative_risk > ip.lbwsg_relative_risk]

,,sex,age,age_start,age_end,iron_fortification_propensity,mother_flour_consumption_propensity,lbwsg_cat,ga_propensity,bw_propensity,gestational_age,birthweight,treatment_deleted_birthweight,mother_is_iron_fortified,mother_daily_flour,birthweight_shift,treated_birthweight,treated_lbwsg_cat,lbwsg_relative_risk
draw,simulant_id,,,,,,,,,,,,,,,,,,
0,914,Male,0.0,0.0,0.019178,0.196692,0.855028,cat32,0.916052,0.981989,39.832104,2490.994255,2490.429737,False,0.0,0.0,2490.429737,cat32,12.059239
50,476,Male,0.0,0.0,0.019178,0.146424,0.766732,cat37,0.796355,0.000677,35.592710,2500.338395,2499.464810,False,0.0,0.0,2499.464810,cat34,28.340447
100,693,Male,0.0,0.0,0.019178,0.177385,0.414875,cat48,0.388551,0.998332,37.388551,3499.165816,3498.766742,False,0.0,0.0,3498.766742,cat48,2.270970
500,464,Male,0.0,0.0,0.019178,0.198901,0.336366,cat44,0.431959,0.995128,38.863917,2997.564209,2996.960629,False,0.0,0.0,2996.960629,cat44,2.642346


In [20]:
ip.loc[bp.lbwsg_relative_risk > ip.lbwsg_relative_risk]

,,sex,age,age_start,age_end,iron_fortification_propensity,mother_flour_consumption_propensity,lbwsg_cat,ga_propensity,bw_propensity,gestational_age,birthweight,treatment_deleted_birthweight,mother_is_iron_fortified,mother_daily_flour,birthweight_shift,treated_birthweight,treated_lbwsg_cat,lbwsg_relative_risk
draw,simulant_id,,,,,,,,,,,,,,,,,,
0,914,Male,0.0,0.0,0.019178,0.196692,0.855028,cat32,0.916052,0.981989,39.832104,2490.994255,2490.429737,True,263.226961,13.167915,2503.597653,cat44,2.880156
50,476,Male,0.0,0.0,0.019178,0.146424,0.766732,cat37,0.796355,0.000677,35.592710,2500.338395,2499.464810,True,210.072548,16.262379,2515.727189,cat37,12.464705
100,693,Male,0.0,0.0,0.019178,0.177385,0.414875,cat48,0.388551,0.998332,37.388551,3499.165816,3498.766742,True,92.338759,3.265482,3502.032225,cat50,2.259545
500,464,Male,0.0,0.0,0.019178,0.198901,0.336366,cat44,0.431959,0.995128,38.863917,2997.564209,2996.960629,True,85.272971,4.560954,3001.521583,cat52,1.410235


### Simulants whose RR went up (opposite of expected):

In [21]:
bp.loc[bp.lbwsg_relative_risk < ip.lbwsg_relative_risk]

sex  age  age_start   age_end  \
draw simulant_id                                     
100  25           Female  0.0        0.0  0.019178   
     456          Female  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
100  25                                0.158926   
     456                               0.194466   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
100  25                                      0.322808     cat54   
     456                                     0.888608     cat45   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
100  25                0.826613       0.998080        39.653227  3999.040127   
     456               0.786616       0.999651        36.786616  3499.825438   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
100  25                             3998.641053                     False   
     456                            3499.426364                     False   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
100  25                          0.0                0.0          3998.641053   
     456                         0.0                0.0          3499.426364   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
100  25                      cat54             1.556346  
     456                     cat45             3.029388

In [22]:
ip.loc[bp.lbwsg_relative_risk < ip.lbwsg_relative_risk]

sex  age  age_start   age_end  \
draw simulant_id                                     
100  25           Female  0.0        0.0  0.019178   
     456          Female  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
100  25                                0.158926   
     456                               0.194466   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
100  25                                      0.322808     cat54   
     456                                     0.888608     cat45   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
100  25                0.826613       0.998080        39.653227  3999.040127   
     456               0.786616       0.999651        36.786616  3499.825438   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
100  25                             3998.641053                      True   
     456                            3499.426364                      True   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
100  25                    84.052688           2.972453          4001.613505   
     456                  283.442239          10.023696          3509.450060   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
100  25                      cat53             1.788830  
     456                     cat47             4.263101

### How many pounds+oz is 3500 g?

7 lb, 11.2 oz

That doesn't seem like a very risky weight, though I guess the gestational age of 36.79 weeks is slightly preterm (<37), so maybe that makes higher weights riskier.

In any case, it looks like for this gestational age, crossing from <3500 g to >3500 g moves you from cat45 to cat47, which increases your relative risk from 3.03 to 4.26 (at least in draw 100).

In [23]:
3.5*2.2

7.700000000000001

In [25]:
.7*16

11.2

## Just to confirm our results, check where categories are different

### Why are there 7 differences instead of 6?

The difference is simulant 241 in draw 500. She went from cat55 to cat56, which are both TMREL, so the RRs are both 1.0.

In [26]:
bp.loc[bp.treated_lbwsg_cat != ip.treated_lbwsg_cat].shape

(7, 18)

In [27]:
bp.loc[bp.treated_lbwsg_cat != ip.treated_lbwsg_cat]

sex  age  age_start   age_end  \
draw simulant_id                                     
0    914            Male  0.0        0.0  0.019178   
50   476            Male  0.0        0.0  0.019178   
100  25           Female  0.0        0.0  0.019178   
     456          Female  0.0        0.0  0.019178   
     693            Male  0.0        0.0  0.019178   
500  241          Female  0.0        0.0  0.019178   
     464            Male  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
0    914                               0.196692   
50   476                               0.146424   
100  25                                0.158926   
     456                               0.194466   
     693                               0.177385   
500  241                               0.189601   
     464                               0.198901   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
0    914                                     0.855028     cat32   
50   476                                     0.766732     cat37   
100  25                                      0.322808     cat54   
     456                                     0.888608     cat45   
     693                                     0.414875     cat48   
500  241                                     0.699834     cat55   
     464                                     0.336366     cat44   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
0    914               0.916052       0.981989        39.832104  2490.994255   
50   476               0.796355       0.000677        35.592710  2500.338395   
100  25                0.826613       0.998080        39.653227  3999.040127   
     456               0.786616       0.999651        36.786616  3499.825438   
     693               0.388551       0.998332        37.388551  3499.165816   
500  241               0.218465       0.995908        40.436930  3997.954136   
     464               0.431959       0.995128        38.863917  2997.564209   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    914                            2490.429737                     False   
50   476                            2499.464810                     False   
100  25                             3998.641053                     False   
     456                            3499.426364                     False   
     693                            3498.766742                     False   
500  241                            3997.350556                     False   
     464                            2996.960629                     False   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    914                         0.0                0.0          2490.429737   
50   476                         0.0                0.0          2499.464810   
100  25                          0.0                0.0          3998.641053   
     456                         0.0                0.0          3499.426364   
     693                         0.0                0.0          3498.766742   
500  241                         0.0                0.0          3997.350556   
     464                         0.0                0.0          2996.960629   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    914                     cat32            12.059239  
50   476                     cat34            28.340447  
100  25                      cat54             1.556346  
     456                     cat45          

In [28]:
ip.loc[bp.treated_lbwsg_cat != ip.treated_lbwsg_cat]

sex  age  age_start   age_end  \
draw simulant_id                                     
0    914            Male  0.0        0.0  0.019178   
50   476            Male  0.0        0.0  0.019178   
100  25           Female  0.0        0.0  0.019178   
     456          Female  0.0        0.0  0.019178   
     693            Male  0.0        0.0  0.019178   
500  241          Female  0.0        0.0  0.019178   
     464            Male  0.0        0.0  0.019178   

                  iron_fortification_propensity  \
draw simulant_id                                  
0    914                               0.196692   
50   476                               0.146424   
100  25                                0.158926   
     456                               0.194466   
     693                               0.177385   
500  241                               0.189601   
     464                               0.198901   

                  mother_flour_consumption_propensity lbwsg_cat  \
draw simulant_id                                                  
0    914                                     0.855028     cat32   
50   476                                     0.766732     cat37   
100  25                                      0.322808     cat54   
     456                                     0.888608     cat45   
     693                                     0.414875     cat48   
500  241                                     0.699834     cat55   
     464                                     0.336366     cat44   

                  ga_propensity  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                               
0    914               0.916052       0.981989        39.832104  2490.994255   
50   476               0.796355       0.000677        35.592710  2500.338395   
100  25                0.826613       0.998080        39.653227  3999.040127   
     456               0.786616       0.999651        36.786616  3499.825438   
     693               0.388551       0.998332        37.388551  3499.165816   
500  241               0.218465       0.995908        40.436930  3997.954136   
     464               0.431959       0.995128        38.863917  2997.564209   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    914                            2490.429737                      True   
50   476                            2499.464810                      True   
100  25                             3998.641053                      True   
     456                            3499.426364                      True   
     693                            3498.766742                      True   
500  241                            3997.350556                      True   
     464                            2996.960629                      True   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    914                  263.226961          13.167915          2503.597653   
50   476                  210.072548          16.262379          2515.727189   
100  25                    84.052688           2.972453          4001.613505   
     456                  283.442239          10.023696          3509.450060   
     693                   92.338759           3.265482          3502.032225   
500  241                  179.933470           9.624013          4006.974569   
     464                   85.272971           4.560954          3001.521583   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    914                     cat44             2.880156  
50   476                     cat37            12.464705  
100  25                      cat53             1.788830  
     456                     cat47          